In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

D:\anaconda\envs\torch-1.13.1-py-3.8\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Get Device for Training

In [3]:
device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
print(f"using {device}")

using cuda


# Define the Class

在 Python 类的定义中，`__init__` 方法是一个特殊的方法，被称为类的构造器或初始化方法。当创建类的新实例时，`__init__` 方法会被自动调用，以初始化新对象的状态。

```python
def __init__(self):
    super().__init__()
```

这段代码中的 `super().__init__()` 调用了父类（超类）的 `__init__` 方法。在面向对象编程中，一个类可以继承自另一个类，`super()` 函数用于调用父类的方法。这行代码的作用是确保父类也被正确地初始化。

### 为什么使用 `super().__init__()`？

1. **父类初始化**：
   - 如果父类中有一些需要初始化的操作（如属性设置、资源分配等），`super().__init__()` 确保这些操作在子类实例化时也被执行。

2. **代码复用**：
   - 使用 `super()` 可以避免在每个子类中重复编写相同的初始化代码，从而提高代码的复用性和可维护性。

3. **维持继承结构**：
   - 在一个继承体系中，`super()` 确保了所有父类都会被正确地初始化，维持了类的继承结构。

### 不加 `super().__init__()` 行不行？

是否需要调用 `super().__init__()` 取决于父类中是否有需要执行的初始化代码。如果父类中的 `__init__` 方法是空的，或者仅仅做一些子类不需要的初始化工作，那么在子类中不调用 `super().__init__()` 也是可以的。

然而，如果父类中的 `__init__` 方法执行了一些必要的初始化操作（如初始化重要的属性或资源），那么在子类中不调用 `super().__init__()` 可能会导致这些必要的初始化被跳过，从而引发运行时错误或不稳定的行为。

### 示例

假设有一个父类 `Parent` 和一个子类 `Child`：

```python
class Parent:
    def __init__(self):
        print("Parent's __init__ is called")

class Child(Parent):
    def __init__(self):
        super().__init__()
        print("Child's __init__ is called")
```

在这个例子中，当创建 `Child` 类的实例时，会先调用 `Parent` 类的 `__init__` 方法，然后再调用 `Child` 类的 `__init__` 方法：

```python
child_instance = Child()
# 输出：
# Parent's __init__ is called
# Child's __init__ is called
```

如果不在 `Child` 类中调用 `super().__init__()`，则只会调用 `Child` 类的 `__init__` 方法，而 `Parent` 类的初始化代码将不会被执行。

总之，`super().__init__()` 是一种良好的编程实践，它确保了父类的初始化代码在子类实例化时被正确执行，有助于维护代码的健壮性和可维护性。


In [23]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )


    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [25]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [52]:
X = torch.rand(1, 28,28, device=device)
logits = model(X)
print(logits.shape, logits)
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab.shape, pred_probab)
y_pred = pred_probab.argmax()
print(f"Predicted class: {y_pred}")

torch.Size([1, 10]) tensor([[-0.0408,  0.1115, -0.0712, -0.0657,  0.1072,  0.0135, -0.0684, -0.0252,
          0.0398, -0.0970]], device='cuda:0', grad_fn=<AddmmBackward0>)
torch.Size([1, 10]) tensor([[0.0967, 0.1126, 0.0938, 0.0943, 0.1121, 0.1021, 0.0941, 0.0982, 0.1048,
         0.0914]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Predicted class: 1


<!-- Model Parameters -->

# Model Parameters

In [56]:
for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0229, -0.0067, -0.0127,  ..., -0.0337,  0.0091, -0.0201],
        [ 0.0116, -0.0164, -0.0231,  ..., -0.0164, -0.0296, -0.0298]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0222, -0.0061], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0200, -0.0060, -0.0222,  ...,  0.0062, -0.0350, -0.0167],
        [-0.0271,  0.0331,  0.0311,  ..., -0.0182, -0.0204, -0.0333]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.bias | Size: torch.Size([512]) | Values : tensor([ 0.0329, -0.0095], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.4.weight | Size: torch.Size([10, 512]) | Values : tensor([[-0.0029, -0.0069, -0.0117,  ...,  0.0286, -0.0289,  0.0275],
        [-0.0323,  0.0325, -0.0084,